<a href="https://colab.research.google.com/github/Jellyjellyjinjin/yolov8x-/blob/main/Yolov8_model_last.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install ultralytics
%pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 10.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import numpy as np
import cv2
import shutil
import yaml
import warnings
warnings.filterwarnings("ignore")

from ultralytics import YOLO
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

WARNING  user config directory is not writeable, defaulting to '/tmp/Ultralytics'.


In [ ]:
SEED = 42
BATCH_SIZE = 8
MODEL = "v8"
data_path = '/content/drive/MyDrive/input'

In [ ]:
import yaml
# 클래스 이름 리스트 생성
with open('/content/drive/MyDrive/classes.txt', 'r') as f:
    lines = f.readlines()
    # Extract class names and store them in a list
    classes = [line.strip().split(',')[1] for line in lines]

root_dir  = "/content/drive/MyDrive/input"     # 이미지, 라벨 상위폴더

yaml_data = {'nc': len(classes),
             'names': classes,
             "path": root_dir,
             "train":os.path.join(root_dir, "train.txt"), # train 경로
             "val":os.path.join(root_dir, "valid.txt") # valid 경로
             }


with open(os.path.join(root_dir, "custom.yaml"), "w") as f:
  yaml.dump(yaml_data, f)

In [ ]:
savepath = '/content/drive/MyDrive/weight_0'

In [ ]:
#model = YOLO(f"{MODEL}/train/weights/last.pt")
model = YOLO("yolov8x") # yolov8n, yolov8s, yolov8m, yolov8l, yolov8x 등이 있음
results = model.train(
    data= '/content/drive/MyDrive/input/custom.yaml',
    imgsz=640,
    epochs=50,
    batch=BATCH_SIZE,
    patience=10,
    workers=16,
    device=0,
    exist_ok=True,
    project=f"{savepath}/yolov8/last",
    name="train",
    cos_lr = True,
    seed=SEED,
    pretrained=False,
    resume=False,
    optimizer="AdamW",
    lr0= 0.0008,
    augment=False,
    val=True,
    cache=True,
    fliplr = 0.0,
    mosaic =1.0,
    mixup = 0.3
    )

Ultralytics YOLOv8.0.116  Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/content/drive/MyDrive/input/custom.yaml, epochs=50, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=True, device=0, workers=16, project=/content/drive/MyDrive/weight_0/yolov8/motionblur, name=train, exist_ok=True, pretrained=False, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True,

In [ ]:
def get_test_image_paths(test_image_paths):
    for i in range(0, len(test_image_paths), BATCH_SIZE):
        yield test_image_paths[i:i+BATCH_SIZE]

In [ ]:
model = YOLO("/content/drive/MyDrive/weight_0/yolov8/motionblur/train/weights/best.pt")
test_image_paths = glob("/content/drive/MyDrive/pretest/images/*.png")
for i, image in tqdm(enumerate(get_test_image_paths(test_image_paths)), total=int(len(test_image_paths)/BATCH_SIZE)):
    model.predict(image, imgsz=(640, 640), iou=0.2, conf=0.3, save_conf=True, save=False, save_txt=True, project=f"{savepath}/yolov8/motionblur/detect", name="predict",
                  exist_ok=True, device=0, augment=True, verbose=False)
    if i % 5 == 0:
        clear_output(wait=True)

 99%|█████████▉| 421/425 [13:54<00:05,  1.45s/it]Results saved to /content/drive/MyDrive/weight_0/yolov8/motionblur/detect/predict
3365 labels saved to /content/drive/MyDrive/weight_0/yolov8/motionblur/detect/predict/labels
 99%|█████████▉| 422/425 [13:56<00:04,  1.44s/it]Results saved to /content/drive/MyDrive/weight_0/yolov8/motionblur/detect/predict
3373 labels saved to /content/drive/MyDrive/weight_0/yolov8/motionblur/detect/predict/labels
100%|█████████▉| 423/425 [13:57<00:02,  1.43s/it]Results saved to /content/drive/MyDrive/weight_0/yolov8/motionblur/detect/predict
3381 labels saved to /content/drive/MyDrive/weight_0/yolov8/motionblur/detect/predict/labels
100%|█████████▉| 424/425 [13:59<00:01,  1.43s/it]Results saved to /content/drive/MyDrive/weight_0/yolov8/motionblur/detect/predict
3389 labels saved to /content/drive/MyDrive/weight_0/yolov8/motionblur/detect/predict/labels
100%|██████████| 425/425 [14:00<00:00,  1.98s/it]


In [ ]:
def yolo_to_labelme(line, image_width, image_height, txt_file_name):
    file_name = txt_file_name.split("/")[-1].replace(".txt", ".png")
    class_id, x, y, width, height, confidence = [float(temp) for temp in line.split()]

    x_min = (x - width / 2) * image_width
    x_max = (x + width / 2) * image_width
    y_min = (y - height / 2) * image_height
    y_max = (y + height / 2) * image_height

    return file_name, int(class_id), confidence, x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max

In [ ]:
infer_txts = glob("/content/drive/MyDrive/weight_0/yolov8/motionblur/detect/predict/labels/*.txt")

results = []
for infer_txt in tqdm(infer_txts):
    base_file_name = infer_txt.split("/")[-1].split(".")[0]
    imgage_height, imgage_width = cv2.imread(f"/content/drive/MyDrive/pretest/images/{base_file_name}.png").shape[:2]
    with open(infer_txt, "r") as reader:
        lines = reader.readlines()
        for line in lines:
            results.append(yolo_to_labelme(line, imgage_width, imgage_height, infer_txt))

df_submission = pd.DataFrame(data=results, columns=["file_name", "class_id", "confidence", "point1_x", "point1_y", "point2_x", "point2_y", "point3_x", "point3_y", "point4_x", "point4_y"])
df_submission.to_csv(f"{savepath}/yolov8/motionblur/result/{MODEL}.csv", index=False)

100%|██████████| 3389/3389 [03:20<00:00, 16.91it/s]
